# Constrained optimization

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

## Basic setup

We consider a smooth objective/cost function $\phi:\mathbb R^n\times \mathbb R^r\to \mathbb R$ that depends on design/control parameters $x$ and random parameters $\xi$:

$$
  \phi(x,\xi)
$$

We cannot directly optimize over $x$ as that function still depends on the random variable $\xi$ which follows a distribution. 
Below, we discuss some aspects of stochastic optimization:

- Scalarization: risk-neutral, risk-averse and robust

For fixed $x$, we can take random draws from the distribution $\phi(x,\xi)$ and plot the resulting as a histogram. This is commonly known as the stochastic average approximation (SAA). 

##  Simple model problem

Let's look at a simple model function where $x=(x_1,x_2)$ and $\xi=(\xi_1,\xi_2)$ is a random variable:

$$
    \phi(x,\xi):= (1+\xi_1)x_1^2 - x_2 - \log(-x_2) + \xi_2^2
$$


In [ ]:
def phi(x1,x2,xi1,xi2):
    "Objective involving uncertainties xi1, xi2"
    return (1+xi1)*x1**2 - x2 - np.log(-x2) + xi2**2


In [ ]:
def plot_phi(x1,x2):
    "Draw histogram for fixed x1,x2"
    no_samp = 100
    XI1 = np.random.uniform(-0.1,0.1,no_samp)
    XI2 = np.random.uniform(-0.5,0.5,no_samp)
    #XI2 = np.random.normal(0,0.3,no_samp)
    P = phi(x1,x2,XI1,XI2)
    P_mean = np.mean(P)
    P_var = np.var(P)
    P_max = np.amax(P)
    print('Pmean:',P_mean)
    print('Pvar:',P_var)
    print('Pmax:',P_max)
    count, bins, ignored = plt.hist(P, 15, density=True)
    plt.xlabel('objective value')
    plt.show()
    
plot_phi(1,-1)


## Objectives for optimization

From this sampled distribution, we can compute various scalar quantities to be used in a minimization over $x$, which we kept fixed above:

For fixed $x$, we can consider:

1. **Risk-neutral** formulation: Take expectation over $\xi$:

    $$
    \min_{x} \mathbb E_{\xi} \phi(x,\xi)
    $$

2. **Risk-averse** formulations: Avoid poor outcomes, e.g.,
    keep variance small:
    
    $$
    \min_{x} \mathbb E_{\xi} \phi(x,\xi) + \mathbb
    V_{\xi}\phi(x,\xi)
    $$
    
3. **Robust** formulations: Minimize worst outcome (no risk!)
    
    $$
    \min_{x} \max_{\xi} \phi(x,\xi)
    $$

Let us implement the risk-neutral objective for our test problem defined above. Since the objective simply amounts to a sum of terms, the gradient can easily be computed by hand.


In [ ]:
 def phi_risk_neutral(x1,x2):
    "Draw histogram for fixed x1,x2"
    no_samp = 100
    np.random.seed(12345)
    XI1 = np.random.uniform(-0.1,0.1,no_samp)
    XI2 = np.random.uniform(-0.5,0.5,no_samp)
    #XI2 = np.random.normal(0,0.3,no_samp)
    P = phi(x1,x2,XI1,XI2)
    P_mean = np.mean(P)
    return P_mean

def phi_risk_neutral_grad(x1,x2):
    "Draw histogram for fixed x1,x2"
    no_samp = 100
    np.random.seed(12345)
    XI1 = np.random.uniform(-0.1,0.1,no_samp)
    XI2 = np.random.uniform(-0.5,0.5,no_samp)
    #XI2 = np.random.normal(0,0.3,no_samp)
    g1 = 2*np.mean(1 + XI1)*x1
    g2 = -1 - 1/x2
    return  np.array([g1,g2])

print(phi_risk_neutral(1,-1))
print(phi_risk_neutral_grad(1,-1))

Now, you have all the information you need to run a deterministic optimization method. 

### Task 1 

Verify gradient using finite differences

### Task 2

Use a steepest descent method (or BFGS) to minimize the risk-neutral function $\phi_{\text{RN}}(x)$.
